### 분석준비

In [1]:
# 필요 라이브러리

# 분석을 위한 라이브러리
import numpy as np
import pandas as pd

# 시각화를 위한 라이브러리
import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime
import os

# 에러메세지 안나오게
import warnings
warnings.filterwarnings('ignore')

# pd.options.display.float_format = '{:.2f}'.format
pd.set_option('display.max_columns',None)
# pd.set_option('display.max_rows',None)

from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px

In [2]:
import matplotlib

matplotlib.rcParams['font.family'] ='Malgun Gothic'

matplotlib.rcParams['axes.unicode_minus'] =False

In [3]:
# 디비 연결
def rds_python_conn():
    import psycopg2
    # rds와 python 연동을 위한 정보 입력
    host = "postgres.c4brhfvhrm5g.ap-northeast-2.rds.amazonaws.com"
    dbname = 'postgres' 
    user = 'postgres' 
    password = "postgres123"
    port = 5432

    connection = psycopg2.connect(host=host,
                                  dbname=dbname ,
                                  user=user ,
                                  password=password ,
                                  port=port)
    return connection

### 유입경로별 체결 건수와 비율

In [4]:
SQL = """
SELECT *
FROM mql m full outer join closed_deals j on m.mql_id=j.mql_id
;
"""
mql = pd.read_sql(SQL, rds_python_conn())
mql

,mql_id,first_contact_date,landing_page_id,origin,mql_id,seller_id,sdr_id,sr_id,won_date,business_segment,lead_type,lead_behaviour_profile,business_type,declared_monthly_revenue
0,5143,2018-02-01,490,social,NaN,NaN,NaN,NaN,NaT,None,None,None,None,NaN
1,5819,2017-10-20,247,paid_search,NaN,NaN,NaN,NaN,NaT,None,None,None,None,NaN
2,1369,2018-03-22,95,organic_search,NaN,NaN,NaN,NaN,NaT,None,None,None,None,NaN
3,312,2018-01-22,435,email,NaN,NaN,NaN,NaN,NaT,None,None,None,None,NaN
4,3690,2018-02-21,119,organic_search,3690.0,2187.0,17.0,2.0,2018-02-26 19:58:54,pet,online_medium,cat,reseller,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,2920,2018-05-22,300,paid_search,NaN,NaN,NaN,NaN,NaT,None,None,None,None,NaN
7996,3895,2018-03-27,269,paid_search,NaN,NaN,NaN,NaN,NaT,None,None,None,None,NaN
7997,7527,2017-08-27,119,organic_search,NaN,NaN,NaN,NaN,NaT,None,None,None,None,NaN
7998,639,2017-10-06,489,organic_search,NaN,NaN,NaN,NaN,NaT,None,None,None,None,NaN


In [5]:
# 체결 날짜의 연도-월 컬럼 생성
mql['won_date_new'] = pd.to_datetime(mql['won_date']).dt.strftime('%Y-%m')

# 컨택한 날짜의 연도-월 컬럼 생성
mql['first_contact_date_new'] = pd.to_datetime(mql['first_contact_date']).dt.strftime('%Y-%m')

In [6]:
won_counts = mql.groupby('origin')['seller_id'].count().sort_values(ascending=False).reset_index()
won_percentages = round(won_counts['seller_id'] / won_counts['seller_id'].sum() * 100, 1)

# make_subplots를 사용하여 서로 다른 유형의 그래프 생성
fig = make_subplots(specs=[[{"secondary_y": True}]])

# 첫 번째 그래프 추가 (막대 그래프)
fig.add_trace(
    go.Bar(
        x=won_counts['origin'],
        y=won_counts['seller_id'],
        name='체결 건수',
        marker_color='#A0BFE0'  # 막대 그래프 색상 지정
    ),
    secondary_y=False
)

# 두 번째 그래프 추가 (라인 플롯)
fig.add_trace(
    go.Scatter(
        x=won_counts['origin'],
        y=won_percentages,
        name='체결 비율',
        mode='lines+markers',
        line=dict(color='#4A55A2'),  # 라인 플롯 색상 지정
        marker=dict(color='#4A55A2')
    ),
    secondary_y=True
)

# 레이아웃 설정
fig.update_layout(
    title='유입경로별 체결 건수와 비율',  # 전체 그래프의 제목
    yaxis=dict(title='체결 건수'),
    yaxis2=dict(title='체결 비율 (%)', overlaying='y', side='right'),
    legend=dict(),
    width=800
)

fig.show()



In [14]:
won_counts = mql.groupby('origin')['seller_id'].count().sort_values(ascending=False).reset_index()

# make_subplots를 사용하여 서로 다른 유형의 그래프 생성
fig = make_subplots(specs=[[{"secondary_y": True}]])

# 첫 번째 그래프 추가 (막대 그래프)
fig.add_trace(
    go.Bar(
        x=won_counts['origin'],
        y=won_counts['seller_id'],
        name='체결 건수',
        marker_color='#4A55A2'  # 막대 그래프 색상 지정
    ),
    secondary_y=False
)

# 레이아웃 설정
fig.update_layout(
    title='유입경로별 체결 건수',  # 전체 그래프의 제목
    xaxis=dict(title='유입경로'),
    yaxis=dict(title='체결 건수'),
    legend=dict(),
    width=800
)

fig.show()



In [37]:
won_percentages = round(won_counts['seller_id'] / won_counts['seller_id'].sum() * 100, 1)

colors = ['#4A55A2','#5D6DAA','#7C8FCC','#9BA1E0','#BAC3F3','#D9D6FF','#C1C6F3','#A9ABE6','#9194D9','#797BBD']
color = ['#4A55A2','#BAC3F3','#BAC3F3','#BAC3F3','#BAC3F3','#BAC3F3','#BAC3F3','#BAC3F3','#BAC3F3','#BAC3F3']
fig = px.pie(values=won_percentages, names=won_counts['origin'])

fig.update_layout(
    title='유입경로별 체결 비율',  # 전체 그래프의 제목
    legend=dict(),
    width=650
)

fig.update_traces(pull=[0.1, 0, 0, 0],marker_colors=color)

fig.show()


In [ ]:
won_percentages = round(won_counts['seller_id'] / won_counts['seller_id'].sum() * 100, 1)

# make_subplots를 사용하여 서로 다른 유형의 그래프 생성
fig = make_subplots(specs=[[{"secondary_y": True}]])

# 첫 번째 그래프 추가 (막대 그래프)
fig.add_trace(
    go.Bar(
        x=won_counts['origin'],
        y=won_counts['seller_id'],
        name='체결 건수',
        marker_color='#A0BFE0'  # 막대 그래프 색상 지정
    ),
    secondary_y=False
)

# 레이아웃 설정
fig.update_layout(
    title='유입경로별 체결 건수와 비율',  # 전체 그래프의 제목
    yaxis=dict(title='체결 건수'),
    yaxis2=dict(title='체결 비율 (%)', overlaying='y', side='right'),
    legend=dict(),
    width=800
)

fig.show()



In [39]:
a = (mql.groupby('origin').size()/mql.shape[0]*100).reset_index(name='mql')
b = (mql[mql['won_date'].notnull()]['origin'].value_counts(normalize=True)*100).reset_index(name='closed_deals')
ab = a.merge(b, on='origin')
ab['difference'] = ab['closed_deals'] - ab['mql']
print(ab)

# make_subplots를 사용하여 서로 다른 유형의 그래프 생성
fig = make_subplots(specs=[[{"secondary_y": True}]])

# 첫 번째 그래프 추가 (라인 플롯)
fig.add_trace(
    go.Scatter(
        x=ab['origin'],
        y=ab['mql'],
        name='잠재고객비율',
        marker_color='#A0BFE0'  # 막대 그래프 색상 지정
    ),
    secondary_y=False
)

# 두 번째 그래프 추가 (라인 플롯)
fig.add_trace(
    go.Scatter(
        x=ab['origin'],
        y=ab['closed_deals'],
        name='체결된 잠재고객비율',
        mode='lines+markers',
        line=dict(color='#7895CB'),  # 라인 플롯 색상 지정
        marker=dict(color='#7895CB')
    )
)
# 세 번째 그래프 추가 (라인 플롯)
fig.add_trace(
    go.Scatter(
        x=ab['origin'],
        y=ab['difference'],
        name='유입경로별 효과',
        mode='lines+markers',
        line=dict(color='#4A55A2'),  # 라인 플롯 색상 지정
        marker=dict(color='#4A55A2')
    )
)

# 레이아웃 설정
fig.update_layout(
    title='유입경로별 체결,미체결된 잠재고객 비율과 효과',  # 전체 그래프의 제목
    yaxis=dict(title='비율'),
    legend=dict(),
    width=800
)

fig.show()

              origin      mql  closed_deals  difference
0     direct_traffic   6.2375      6.650831    0.413331
1            display   1.4750      0.712589   -0.762411
2              email   6.1625      1.781473   -4.381027
3     organic_search  28.7000     32.185273    3.485273
4              other   1.8750      0.475059   -1.399941
5  other_publicities   0.8125      0.356295   -0.456205
6        paid_search  19.8250     23.159145    3.334145
7           referral   3.5500      2.850356   -0.699644
8             social  16.8750      8.907363   -7.967637
9            unknown  14.4875     22.921615    8.434115


### 매출파악(구매자 결제건)

In [55]:
# ('delivered','shipped','invoiced','processing') 이경우만 가지고 파악.
SQL = """
SELECT *
FROM "order"
WHERE order_status in ('delivered','shipped','invoiced','processing') and order_purchase_timestamp < '2018-09-01'
order by order_id
"""


order = pd.read_sql(SQL, rds_python_conn())
order

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,0,10071,delivered,2017-02-13 16:12:43,2017-02-13 16:25:09,2017-02-14 10:58:44,2017-03-01 11:16:06,2017-03-24
1,1,36159,delivered,2018-01-07 21:13:42,2018-01-07 21:27:12,2018-01-10 18:36:40,2018-01-15 20:10:13,2018-01-31
2,2,23371,delivered,2018-08-23 13:56:50,2018-08-23 14:10:15,2018-08-23 15:42:00,2018-08-27 18:32:55,2018-09-04
3,3,31171,delivered,2017-12-12 08:39:09,2017-12-12 10:32:39,2017-12-13 18:38:53,2017-12-18 20:09:12,2018-01-11
4,4,53810,delivered,2017-11-08 15:02:42,2017-11-09 15:11:00,2017-11-16 19:34:27,2017-11-21 12:52:26,2017-11-22
...,...,...,...,...,...,...,...,...
98194,99436,27340,delivered,2018-05-08 19:42:13,2018-05-08 20:31:59,2018-05-09 12:39:00,2018-05-16 23:38:52,2018-05-22
98195,99437,87752,delivered,2017-07-05 11:56:09,2017-07-05 17:43:11,2017-07-11 16:34:32,2017-07-19 20:34:51,2017-08-04
98196,99438,92997,delivered,2017-07-05 18:06:31,2017-07-05 18:23:31,2017-07-07 13:49:57,2017-07-10 19:15:30,2017-07-21
98197,99439,97562,delivered,2017-10-27 12:07:08,2017-10-27 12:28:41,2017-10-30 13:21:46,2017-11-14 17:47:07,2017-11-21


In [56]:
SQL = """
SELECT customer_id, customer_unique_id
FROM customers
"""


uniqueid = pd.read_sql(SQL, rds_python_conn())
uniqueid

,customer_id,customer_unique_id
0,93596,86343
1,96387,55527
2,71593,44820
3,17589,36016
4,46972,96049
...,...,...
99436,40421,29126
99437,65930,3710
99438,83443,38830
99439,52687,79036


In [57]:
uniqueid.nunique()

customer_id           99441
customer_unique_id    96096
dtype: int64

In [58]:
order_uniqueid = order.merge(uniqueid,on='customer_id',how='left')
order_uniqueid

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_unique_id
0,0,10071,delivered,2017-02-13 16:12:43,2017-02-13 16:25:09,2017-02-14 10:58:44,2017-03-01 11:16:06,2017-03-24,21081
1,1,36159,delivered,2018-01-07 21:13:42,2018-01-07 21:27:12,2018-01-10 18:36:40,2018-01-15 20:10:13,2018-01-31,9745
2,2,23371,delivered,2018-08-23 13:56:50,2018-08-23 14:10:15,2018-08-23 15:42:00,2018-08-27 18:32:55,2018-09-04,53790
3,3,31171,delivered,2017-12-12 08:39:09,2017-12-12 10:32:39,2017-12-13 18:38:53,2017-12-18 20:09:12,2018-01-11,60982
4,4,53810,delivered,2017-11-08 15:02:42,2017-11-09 15:11:00,2017-11-16 19:34:27,2017-11-21 12:52:26,2017-11-22,77280
...,...,...,...,...,...,...,...,...,...
98194,99436,27340,delivered,2018-05-08 19:42:13,2018-05-08 20:31:59,2018-05-09 12:39:00,2018-05-16 23:38:52,2018-05-22,84335
98195,99437,87752,delivered,2017-07-05 11:56:09,2017-07-05 17:43:11,2017-07-11 16:34:32,2017-07-19 20:34:51,2017-08-04,33367
98196,99438,92997,delivered,2017-07-05 18:06:31,2017-07-05 18:23:31,2017-07-07 13:49:57,2017-07-10 19:15:30,2017-07-21,48993
98197,99439,97562,delivered,2017-10-27 12:07:08,2017-10-27 12:28:41,2017-10-30 13:21:46,2017-11-14 17:47:07,2017-11-21,69845


In [81]:
# ('delivered','shipped','invoiced','processing') 이경우만 가지고 파악.
SQL = """
SELECT *
FROM "order" o join payments p on o.order_id=p.order_Id 
WHERE order_status in ('delivered','shipped','invoiced','processing') and order_purchase_timestamp < '2018-09-01' am
order by o.order_id
"""


orderpayment = pd.read_sql(SQL, rds_python_conn())
orderpayment = orderpayment.loc[:, ~orderpayment.columns.duplicated()]
orderpayment

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,payment_sequential,payment_type,payment_installments,payment_value
0,0,10071,delivered,2017-02-13 16:12:43,2017-02-13 16:25:09,2017-02-14 10:58:44,2017-03-01 11:16:06,2017-03-24,1,credit_card,2,104.87
1,1,36159,delivered,2018-01-07 21:13:42,2018-01-07 21:27:12,2018-01-10 18:36:40,2018-01-15 20:10:13,2018-01-31,1,credit_card,3,126.54
2,2,23371,delivered,2018-08-23 13:56:50,2018-08-23 14:10:15,2018-08-23 15:42:00,2018-08-27 18:32:55,2018-09-04,1,credit_card,8,279.11
3,3,31171,delivered,2017-12-12 08:39:09,2017-12-12 10:32:39,2017-12-13 18:38:53,2017-12-18 20:09:12,2018-01-11,1,credit_card,7,73.64
4,4,53810,delivered,2017-11-08 15:02:42,2017-11-09 15:11:00,2017-11-16 19:34:27,2017-11-21 12:52:26,2017-11-22,1,credit_card,2,24.68
...,...,...,...,...,...,...,...,...,...,...,...,...
102560,99436,27340,delivered,2018-05-08 19:42:13,2018-05-08 20:31:59,2018-05-09 12:39:00,2018-05-16 23:38:52,2018-05-22,1,credit_card,10,1057.88
102561,99437,87752,delivered,2017-07-05 11:56:09,2017-07-05 17:43:11,2017-07-11 16:34:32,2017-07-19 20:34:51,2017-08-04,1,credit_card,1,264.66
102562,99438,92997,delivered,2017-07-05 18:06:31,2017-07-05 18:23:31,2017-07-07 13:49:57,2017-07-10 19:15:30,2017-07-21,1,credit_card,1,27.77
102563,99439,97562,delivered,2017-10-27 12:07:08,2017-10-27 12:28:41,2017-10-30 13:21:46,2017-11-14 17:47:07,2017-11-21,1,credit_card,1,61.05


In [82]:
# 객단가
# 가설 1 : 매출이 감소한건 객단가가 감소했기 때문이다.
# 전체기간기준
print(orderpayment['payment_value'].sum())
orderpayment['payment_value'].sum()/orderpayment.shape[0]

15738041.370000001


153.44456071759373

In [154]:
# (orderpayment.groupby(pd.to_datetime(orderpayment['order_purchase_timestamp']).dt.strftime('%Y-%m'))['payment_value'].sum()/orderpayment.groupby(pd.to_datetime(orderpayment['order_purchase_timestamp']).dt.strftime('%Y-%m'))['order_id'].size()).plot(label='연도-월별 객단가')
# plt.axhline(y=orderpayment['payment_value'].sum()/orderpayment.shape[0], color='gray', linestyle='--', label='전체객단가평균')
# plt.title('2018년-월별 객단가')
# plt.xlabel('연도-월')
# plt.ylabel('객단가')

# plt.legend()


In [84]:
a = order_uniqueid.groupby(pd.to_datetime(order_uniqueid['order_purchase_timestamp']).dt.strftime('%Y-%m'))['customer_unique_id'].size().reset_index()

# 첫 번째 그래프 추가 (라인 플롯)
fig = px.line(
        x=a['order_purchase_timestamp'],
        y=a['customer_unique_id'],
    )

# 라인 스타일과 색상 설정
fig.update_traces(
    line=dict(color='#4A55A2', dash='solid'),  # 선 스타일과 색상 설정
    marker=dict(color='#4A55A2'),  # 마커 색상 설정
)

# 레이아웃 설정
fig.update_layout(
    title='연도-월별 고객의 주문 건수',  # 전체 그래프의 제목
    xaxis=dict(title='날짜'),  # x축 레이블 설정
    yaxis=dict(title='주문 건수'),
    legend=dict(),
    width=800
)

fig.show()

In [119]:
a = orderpayment.groupby(pd.to_datetime(orderpayment['order_purchase_timestamp']).dt.strftime('%Y-%m'))['payment_value'].sum().reset_index()

# 첫 번째 그래프 추가 (라인 플롯)
fig = px.line(
        x=a['order_purchase_timestamp'],
        y=a['payment_value'],
    )

# 라인 스타일과 색상 설정
fig.update_traces(
    line=dict(color='#4A55A2', dash='solid'),  # 선 스타일과 색상 설정
    marker=dict(color='#4A55A2'),  # 마커 색상 설정
)

# 레이아웃 설정
fig.update_layout(
    title='연도-월별 매출',  # 전체 그래프의 제목
    xaxis=dict(title='날짜'),  # x축 레이블 설정
    yaxis=dict(title='매출'),
    legend=dict(),
    width=800
)

fig.show()

In [117]:
SQL = """
SELECT *
FROM "order" o join payments p on o.order_id=p.order_Id 
WHERE order_status in ('delivered','shipped','invoiced','processing') and order_purchase_timestamp < '2018-09-01'
order by o.order_id
"""


orderpayment = pd.read_sql(SQL, rds_python_conn())
orderpayment = orderpayment.loc[:, ~orderpayment.columns.duplicated()]
orderpayment

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,payment_sequential,payment_type,payment_installments,payment_value
0,0,10071,delivered,2017-02-13 16:12:43,2017-02-13 16:25:09,2017-02-14 10:58:44,2017-03-01 11:16:06,2017-03-24,1,credit_card,2,104.87
1,1,36159,delivered,2018-01-07 21:13:42,2018-01-07 21:27:12,2018-01-10 18:36:40,2018-01-15 20:10:13,2018-01-31,1,credit_card,3,126.54
2,2,23371,delivered,2018-08-23 13:56:50,2018-08-23 14:10:15,2018-08-23 15:42:00,2018-08-27 18:32:55,2018-09-04,1,credit_card,8,279.11
3,3,31171,delivered,2017-12-12 08:39:09,2017-12-12 10:32:39,2017-12-13 18:38:53,2017-12-18 20:09:12,2018-01-11,1,credit_card,7,73.64
4,4,53810,delivered,2017-11-08 15:02:42,2017-11-09 15:11:00,2017-11-16 19:34:27,2017-11-21 12:52:26,2017-11-22,1,credit_card,2,24.68
...,...,...,...,...,...,...,...,...,...,...,...,...
102560,99436,27340,delivered,2018-05-08 19:42:13,2018-05-08 20:31:59,2018-05-09 12:39:00,2018-05-16 23:38:52,2018-05-22,1,credit_card,10,1057.88
102561,99437,87752,delivered,2017-07-05 11:56:09,2017-07-05 17:43:11,2017-07-11 16:34:32,2017-07-19 20:34:51,2017-08-04,1,credit_card,1,264.66
102562,99438,92997,delivered,2017-07-05 18:06:31,2017-07-05 18:23:31,2017-07-07 13:49:57,2017-07-10 19:15:30,2017-07-21,1,credit_card,1,27.77
102563,99439,97562,delivered,2017-10-27 12:07:08,2017-10-27 12:28:41,2017-10-30 13:21:46,2017-11-14 17:47:07,2017-11-21,1,credit_card,1,61.05


In [151]:
lis = []
name=[]
a = orderpayment.groupby(pd.to_datetime(orderpayment['order_purchase_timestamp']).dt.strftime('%Y-%m'))['payment_value'].sum()
for i in range(len(a)-1):
    lis.append(round((a[i+1]-a[i])/a[i]*100,1))
    name.append(list(a.index)[i+1])
lis_name = pd.DataFrame({'lis':lis,
                        'name':name})

In [153]:
fig = px.line(
        lis_name[7:],
        x='name',
        y='lis'
    )

# 라인 스타일과 색상 설정
fig.update_traces(
    line=dict(color='#4A55A2'),  # 선 스타일과 색상 설정
    marker=dict(color='#4A55A2'),  # 마커 색상 설정
)

# 레이아웃 설정
fig.update_layout(
    title='매출의 전월 대비 증가율',  # 전체 그래프의 제목
    xaxis=dict(title='날짜'),  # x축 레이블 설정
    yaxis=dict(title='증가율', range=[-40, 60]),  # y축 범위 설정
    legend=dict(),
    width=800
)

fig.show()